In [ ]:
import pandas as pd 
import pickle
import datetime

# Creatinine

In [ ]:
df = pd.read_csv('/projects/data/GS/HF/basic_obs.csv') # loading the lab test results data

In [ ]:
print(df['basicobs_itemname_analysed'].unique())
df['basicobs_itemname_analysed_b'] = df['basicobs_itemname_analysed'].str.upper() 

In [ ]:
creatinine = df.loc[df['basicobs_itemname_analysed_b'].str.contains('CREATININE', case=False)] # keeping serum creatinine test results

In [ ]:
creatinine['basicobs_unitofmeasure'].unique()

In [ ]:
hf = pd.read_csv('/projects/RALES TRIAL/1A/notspiro dataextraction/notspiro data/hf_patients.csv')  
len(hf)  # loading HF patients

In [ ]:
creatinine = pd.merge(creatinine, hf, on='client_idcode', how='right') 
creatinine = creatinine[creatinine['basicobs_value_numeric'].notna()] # removing rows with no test results

In [ ]:
creatinine.info()

In [ ]:
del creatinine['Unnamed: 0']
creatinine['basicobs_createdwhen'] = pd.to_datetime(creatinine['basicobs_createdwhen'], utc=True).dt.date 

In [ ]:
%%time 
creatinine['difference'] = pd.to_datetime(creatinine['start_date']) - pd.to_datetime(creatinine['basicobs_createdwhen'])
min_time = datetime.timedelta(days=1) 
max_time = datetime.timedelta(days=183)
t1 = creatinine['difference'] >= min_time 
t2 = creatinine['difference'] <= max_time
in_window = t1 & t2 
creatinine['in_window'] = in_window

In [ ]:
creatinine = creatinine.loc[creatinine['in_window'] == True]
creatinine.shape

In [ ]:
creatinine.head()

In [ ]:
creatinine.loc[creatinine['client_idcode'] == 'P744081'] # checking

In [ ]:
# keeping the closest serum creatinine test result to the HF diagnosis date

creatinine = creatinine[creatinine.groupby('client_idcode').basicobs_createdwhen.transform('max') == creatinine['basicobs_createdwhen']]
duplicates = creatinine['client_idcode'].duplicated()
duplicates.value_counts()

In [ ]:
creatinine.head()

In [ ]:
 creatinine['result'] = creatinine.groupby('client_idcode')['basicobs_value_numeric'].transform('mean')

In [ ]:
creatinine.drop_duplicates(subset='client_idcode', keep='first', inplace = True)
del creatinine['basicobs_value_numeric']
creatinine.rename(columns={'result':'basicobs_value_numeric'}, inplace=True)
creatinine.reset_index(drop=True, inplace=True)

In [ ]:
duplicates = creatinine['client_idcode'].duplicated()
duplicates.value_counts()

In [ ]:
creatinine.basicobs_value_numeric.isnull().sum()

In [ ]:
creatinine_dict = dict(zip(creatinine.client_idcode, creatinine.basicobs_value_numeric))
print(len(creatinine_dict.keys()))

In [ ]:
with open ('/projects/RALES TRIAL/1A/notspiro dataextraction/notspiro data/creatinine.pickle', 'wb') as f:
    pickle.dump(creatinine_dict,f)

# Potassium

In [ ]:
potassium = df.loc[df['basicobs_itemname_analysed_b'].str.contains('POTASSIUM', case=False)]

In [ ]:
potassium['basicobs_unitofmeasure'].unique()

In [ ]:
potassium = pd.merge(potassium, hf, on='client_idcode', how='right') 
potassium = potassium[potassium['basicobs_value_numeric'].notna()] 

In [ ]:
potassium.info()

In [ ]:
del potassium['Unnamed: 0']
potassium['basicobs_createdwhen'] = pd.to_datetime(potassium['basicobs_createdwhen'], utc=True).dt.date 

In [ ]:
%%time 
potassium['difference'] = pd.to_datetime(potassium['start_date']) - pd.to_datetime(potassium['basicobs_createdwhen'])
min_time = datetime.timedelta(days=1) 
max_time = datetime.timedelta(days=183)
t1 = potassium['difference'] >= min_time 
t2 = potassium['difference'] <= max_time
in_window = t1 & t2 
potassium['in_window'] = in_window

In [ ]:
potassium = potassium.loc[potassium['in_window'] == True]
potassium.shape

In [ ]:
potassium = potassium[potassium.groupby('client_idcode').basicobs_createdwhen.transform('max') == potassium['basicobs_createdwhen']]
duplicates = potassium['client_idcode'].duplicated()
duplicates.value_counts()

In [ ]:
potassium['result'] = potassium.groupby('client_idcode')['basicobs_value_numeric'].transform('mean')
potassium.drop_duplicates(subset='client_idcode', keep='first', inplace = True)
del potassium['basicobs_value_numeric']
potassium.rename(columns={'result':'basicobs_value_numeric'}, inplace=True)
potassium.reset_index(drop=True, inplace=True)

In [ ]:
duplicates = potassium['client_idcode'].duplicated()
duplicates.value_counts()

In [ ]:
potassium.basicobs_value_numeric.isnull().sum()

In [ ]:
potassium_dict = dict(zip(potassium.client_idcode, potassium.basicobs_value_numeric))
len(potassium_dict)

In [ ]:
with open ('/projects/RALES TRIAL/1A/notspiro dataextraction/notspiro data/potassium.pickle', 'wb') as f:
    pickle.dump(potassium_dict,f)